<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_p25/blob/main/professor/numpy/notebooks/08_IO_Pickle_JSON_NumPy.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 08 — Entrada/Salida: Pickle, JSON y NumPy (profundizado)

Objetivo: entender a fondo cómo guardar y cargar datos con Python y NumPy, cuándo usar cada formato y sus implicaciones de legibilidad, tamaño, portabilidad y seguridad.

¿Qué veremos?
- Pickle: qué es, cómo funciona (a alto nivel), protocolos, riesgos de seguridad, cuándo usarlo.
- JSON: formato legible por humanos; `indent`, `ensure_ascii`, `sort_keys`, tamaños y limitaciones.
- NumPy: `np.save`, `np.load`, `np.savez`, `np.savez_compressed` para arreglos eficientes.
- Serialización de objetos NumPy en JSON: por qué falla directo y cómo hacerlo bien.
- Demostraciones desde la terminal con `!ls`, `!file`, `!head`, `!tail` para inspección.

Regla rápida:
- ¿Necesitas legibilidad humana y compatibilidad? JSON.
- ¿Necesitas arreglos NumPy grandes y eficientes? Formatos NumPy.
- ¿Necesitas serializar objetos Python arbitrarios (no seguros)? Pickle, con cuidado.


### Qué hace esta celda (ndarray + pickle)

- Crea un `ndarray` y lo guarda en `array.pkl` con `pickle`.
- Modifica el arreglo original para demostrar que el archivo conserva el estado anterior.
- Carga el pickle y verifica que se recupera el arreglo guardado.
- Recalca: el pickle es binario y no legible por humanos, pero guarda estructura/dtype/shape.



In [32]:
import numpy as np, pickle, json, os, sys

# Pickle: qué es y cómo funciona (alto nivel)
# - Serializa objetos Python a un flujo binario (no legible por humanos).
# - Usa "protocolos" (versiones); por defecto, protocolo más alto disponible.
# - Riesgo: cargar (`pickle.load`) puede ejecutar código arbitrario. No cargues archivos no confiables.

# Guardar y cargar con pickle (ndarray)
arr = np.array([1, 2, 3, 4, 5])
with open("array.pkl", "wb") as f:
    pickle.dump(arr, f, protocol=pickle.HIGHEST_PROTOCOL)

arr[0] = -1
with open("array.pkl", "rb") as f:
    loaded = pickle.load(f)
loaded


array([1, 2, 3, 4, 5])

### Qué hace esta celda (dict + pickle)

- Serializa un diccionario con `pickle` usando el protocolo más alto.
- El archivo resultante es binario y no legible por humanos.
- Se decodifica con `pickle.load` para reconstruir el objeto original.
- Precaución: no cargues pickles de fuentes no confiables (riesgo de ejecución de código).



In [33]:
# Dict + pickle (binario no legible)
data = {"name": "John Doe", "age": 32, "email": "johndoe@example.com"}
with open("data.pkl", "wb") as f:
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
with open("data.pkl", "rb") as f:
    data_loaded = pickle.load(f)
data_loaded


{'name': 'John Doe', 'age': 32, 'email': 'johndoe@example.com'}

### Qué hace esta celda (JSON legible)

- Escribe el `dict` en JSON con `indent=2` (más legible), `sort_keys=True` (orden de claves determinista) y `ensure_ascii=False` (caracteres tal cual).
- Lee de vuelta para comprobar la sintaxis y la decodificación.
- Recuerda: más legibilidad → mayor tamaño físico del archivo.



In [34]:
# JSON legible (humano): indent, sort_keys, UTF-8
with open("data.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False, sort_keys=True)
with open("data.json", "r", encoding="utf-8") as f:
    json.load(f)


### Qué hace esta celda (NumPy + JSON)

- Muestra que `json.dumps(np.array(...))` falla porque `ndarray` no es JSON serializable.
- Define una función `default` que convierte `ndarray` a listas y `np.generic` a escalares de Python.
- Guarda un `ndarray` en JSON usando `default` y lo lee de vuelta.
- Nota: este enfoque pierde `dtype`/`shape` (se recuperan como lista y tipos Python). Más abajo hay un round‑trip con metadatos.



In [35]:
# JSON + NumPy: falla directo, usar default para serializar
# Edge case: ndarray/np.number no son JSON serializables por defecto
try:
    json.dumps(np.array([1, 2, 3]))
except TypeError as e:
    print("Error esperado:", e)

# Encoder: convertir ndarray -> list, np.number -> Python scalar

def json_serializable(o):
    if isinstance(o, np.ndarray):
        return o.tolist()
    if isinstance(o, np.generic):  # cubre np.number, np.int64, etc.
        return o.item()
    raise TypeError(f"No serializable: {type(o)}")

arr2 = np.array([[1, 2, 3], [4, 5, 6]], dtype=np.int64)
with open("ndarray_json.json", "w", encoding="utf-8") as f:
    json.dump(arr2, f, default=json_serializable)
with open("ndarray_json.json", "r", encoding="utf-8") as f:
    json.load(f)


Error esperado: Object of type ndarray is not JSON serializable


### Qué hace esta celda (UTF‑8 y caracteres)

- Escribe una cadena con caracteres no ASCII (Ñ, acentos y kanji) en JSON.
- Usa `ensure_ascii=False` para mantener los caracteres tal cual (sin escapes `\uXXXX`).
- Lee de vuelta para verificar que la codificación UTF‑8 funciona correctamente.



In [36]:
# UTF-8 y caracteres especiales
with open("string.json", "w", encoding="utf-8") as f:
    json.dump("Ñ á é 漢字", f, ensure_ascii=False)
with open("string.json", "r", encoding="utf-8") as f:
    json.load(f)


## Pickle en detalle: protocolos y seguridad

- Formato binario específico de Python: no legible por humanos ni portable fuera de Python.
- Protocolos: versiones de serialización (mayor protocolo = mejor rendimiento/compatibilidad con tipos nuevos). Usa `pickle.HIGHEST_PROTOCOL`.
- Seguridad: `pickle.load` puede ejecutar código arbitrario. Solo carga archivos de fuentes confiables. Alternativas seguras: JSON (legible) o formatos NumPy.
- Portabilidad: pickles viejos pueden fallar en versiones nuevas o con cambios de clases.



### Qué hace esta celda (inspección con shell)

- Lista archivos generados y tamaños (`!ls -lh`).
- Muestra tipo de archivo detectado por el sistema (`!file`), evidenciando que Pickle es binario y JSON es texto.
- Enseña por qué no tiene sentido usar `head` en binarios (bytes no legibles), y muestra `head` de un JSON como ejemplo de legibilidad.



In [37]:

# Inspección extendida con shell: tamaños, tipos, head/tail y strings
!ls -lh array.pkl data.pkl data.json ndarray_json.json string.json
!file array.pkl data.pkl data.json ndarray_json.json string.json

print("\n--- Print head del pkl directo: array.pkl:  ---")
!head array.pkl

# Binarios: usar hexdump para ver bytes (no legible por humanos)
print("\n--- array.pkl: hexdump head (64 bytes) ---")
!hexdump -C -n 64 array.pkl | head -n 10
print("\n--- array.pkl: hexdump tail (64 bytes) ---")
!tail -c 64 array.pkl | hexdump -C
print("\n--- array.pkl: strings (primeras 5 líneas) ---")
!bash -lc 'command -v strings >/dev/null 2>&1 && strings -n 4 array.pkl | head -n 5 || echo "strings no disponible"'

print("\n--- data.pkl: hexdump head (64 bytes) ---")
!hexdump -C -n 64 data.pkl | head -n 10
print("\n--- data.pkl: hexdump tail (64 bytes) ---")
!tail -c 64 data.pkl | hexdump -C
print("\n--- data.pkl: strings (primeras 5 líneas) ---")
!bash -lc 'command -v strings >/dev/null 2>&1 && strings -n 4 data.pkl | head -n 5 || echo "strings no disponible"'

# JSON (texto): head/tail muestran contenido legible
print("\n--- data.json (head) ---")
!head -n 5 data.json
print("\n--- data.json (tail) ---")
!tail -n 5 data.json

print("\n--- ndarray_json.json (head) ---")
!head -n 5 ndarray_json.json
print("\n--- ndarray_json.json (tail) ---")
!tail -n 5 ndarray_json.json

print("\n--- string.json (head) ---")
!head -n 5 string.json
print("\n--- string.json (tail) ---")
!tail -n 5 string.json


-rw-rw-r-- 1 uumami uumami 167 Oct 22 18:13 array.pkl
-rw-rw-r-- 1 uumami uumami  71 Oct 22 18:13 data.json
-rw-rw-r-- 1 uumami uumami  72 Oct 22 18:13 data.pkl
-rw-rw-r-- 1 uumami uumami  22 Oct 22 18:13 ndarray_json.json
-rw-rw-r-- 1 uumami uumami  17 Oct 22 18:13 string.json
array.pkl:         XENIX 8086 relocatable or 80286 small model
data.pkl:          XENIX 8086 relocatable or 80286 small model
data.json:         JSON data
ndarray_json.json: JSON data
string.json:       Unicode text, UTF-8 text, with no line terminators

--- Print head del pkl directo: array.pkl:  ---
���       �numpy._core.numeric��_frombuffer���(�(                                          ��numpy��dtype����i8�����R�(K�<�NNNJ����J����K t�bK���C�t�R�.
--- array.pkl: hexdump head (64 bytes) ---
00000000  80 05 95 9c 00 00 00 00  00 00 00 8c 13 6e 75 6d  |.............num|
00000010  70 79 2e 5f 63 6f 72 65  2e 6e 75 6d 65 72 69 63  |py._core.numeric|
00000020  94 8c 0b 5f 66 72 6f 6d  62 75 66 66 65

## JSON en detalle: legibilidad y tamaños

- Legible por humanos y ampliamente interoperable.
- `indent` mejora legibilidad; `sort_keys` hace determinista el orden de claves.
- `ensure_ascii=False` preserva caracteres no ASCII (útil en español y otros idiomas).
- Tamaño: añadir `indent` y espacios aumenta el tamaño del archivo; úsalos cuando la legibilidad importe.



In [38]:
# Comparar tamaños: JSON compacto vs legible
small = {"z": 1, "á": "hola", "lista": list(range(10))}
with open("compact.json", "w", encoding="utf-8") as f:
    json.dump(small, f, ensure_ascii=False, separators=(",", ":"))
with open("pretty.json", "w", encoding="utf-8") as f:
    json.dump(small, f, ensure_ascii=False, indent=2, sort_keys=True)
!ls -lh compact.json pretty.json
!wc -c compact.json pretty.json


-rw-rw-r-- 1 uumami uumami  49 Oct 22 18:13 compact.json
-rw-rw-r-- 1 uumami uumami 115 Oct 22 18:13 pretty.json
 49 compact.json
115 pretty.json
164 total


## Round‑trip JSON para `ndarray`: preservar `dtype` y `shape`

Guardar como lista pierde la información de `dtype` y `shape`. Podemos incluir metadatos y reconstruir con `object_hook`.



### Qué hace esta celda (Round‑trip JSON para `ndarray` con metadatos)

Esta celda demuestra cómo guardar y recuperar arreglos de NumPy en JSON sin perder `dtype` ni `shape`, algo que se pierde si solo convertimos a listas.

- Se define `NumpyEncoder`, un `JSONEncoder` personalizado que, cuando ve un `np.ndarray`, lo convierte a un `dict` con:
  - `__ndarray__`: marca para reconocer que este `dict` representa un arreglo.
  - `dtype`: el tipo (como string), por ejemplo `"float32"`.
  - `shape`: la forma, por ejemplo `(3, 4)`.
  - `data`: los datos como lista anidada (JSON no admite binarios ni `ndarray`).
  También convierte `np.generic` (p. ej. `np.float32(1.0)`) a escalares de Python con `.item()`.

- Se define `numpy_object_hook`, una función que `json.load` invoca para cada `dict` leído. Si encuentra la marca `__ndarray__`, reconstruye el arreglo con `np.array(..., dtype=...)` y lo re‑da forma con `reshape(shape)`.

- Se crea `orig` con un `ndarray` y metadatos; se guarda con `json.dump(..., cls=NumpyEncoder, ...)` y se carga con `json.load(..., object_hook=numpy_object_hook)`.

- Aserciones: comprueban que lo cargado es `np.ndarray`, que conserva `dtype` y `shape`, y que los valores coinciden con `np.allclose`.

Notas prácticas:
- Este método es portable y legible (JSON), pero los archivos pueden ser grandes y lentos para arreglos muy grandes.
- Es “permisisvo”/seguro comparado con Pickle (no ejecuta código), a costa de más tamaño/CPU.
- Para datos numéricos grandes, prefiere `np.save/np.load` o `np.savez(_compressed)`; usa este esquema JSON cuando necesites interoperabilidad y legibilidad.



In [39]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, np.ndarray):
            return {
                "__ndarray__": True,
                "dtype": str(o.dtype),
                "shape": o.shape,
                "data": o.tolist(),
            }
        if isinstance(o, np.generic):
            return o.item()
        return super().default(o)

def numpy_object_hook(d):
    if d.get("__ndarray__"):
        arr = np.array(d["data"], dtype=np.dtype(d["dtype"]))
        return arr.reshape(d["shape"])
    return d

orig = {
    "mat": np.arange(12, dtype=np.float32).reshape(3, 4),
    "meta": {"name": "demo"},
}

with open("arr_with_meta.json", "w", encoding="utf-8") as f:
    json.dump(orig, f, cls=NumpyEncoder, ensure_ascii=False, indent=2)

with open("arr_with_meta.json", "r", encoding="utf-8") as f:
    loaded = json.load(f, object_hook=numpy_object_hook)

assert isinstance(loaded["mat"], np.ndarray)
assert loaded["mat"].dtype == np.float32
assert loaded["mat"].shape == (3, 4)
np.allclose(loaded["mat"], orig["mat"])


True

## Edge cases en JSON: NaN/Inf y datetime

- JSON estándar no tiene `NaN`/`Infinity`: Python permite `NaN` por defecto (`allow_nan=True`), pero otros parsers podrían fallar. Mejor mapear a `null` o strings.
- Fechas: no hay tipo nativo; usa `ISO 8601` (string) y reconstruye con `object_hook`.



In [40]:
import math, datetime

bad = {"nan": float("nan"), "inf": float("inf")}

# Por defecto, Python permite NaN/Inf, pero no es JSON estándar
s = json.dumps(bad)  # allow_nan=True por defecto
print("JSON con NaN/Inf (no estándar):", s)

# Mapeo seguro: convertir a null
safe = {k: None if isinstance(v, float) and (math.isnan(v) or math.isinf(v)) else v for k, v in bad.items()}
print("Seguro:", json.dumps(safe))

# Datetime a ISO 8601
now = datetime.datetime.now().astimezone()
rec = {"timestamp": now.isoformat()}
print("Datetime ISO:", json.dumps(rec))


JSON con NaN/Inf (no estándar): {"nan": NaN, "inf": Infinity}
Seguro: {"nan": null, "inf": null}
Datetime ISO: {"timestamp": "2025-10-22T18:13:27.053653-06:00"}


## Formatos de NumPy: `npy` y `npz`

Para arreglos grandes, usa formatos nativos de NumPy: rápidos y compactos.
- `np.save`/`np.load` guardan/cargan un solo arreglo `.npy` (contiene dtype y shape).
- `np.savez` guarda varios arreglos en `.npz` (zip sin compresión); `np.savez_compressed` aplica compresión (más pequeño, puede ser más lento al guardar/cargar).



In [41]:
A = np.random.rand(1000, 1000).astype(np.float32)
B = np.random.rand(1000, 1000).astype(np.float32)
np.save("A.npy", A)
np.savez("AB.npz", A=A, B=B)
np.savez_compressed("AB_comp.npz", A=A, B=B)
!ls -lh A.npy AB.npz AB_comp.npz

# Carga y verificación
A2 = np.load("A.npy")
with np.load("AB.npz") as z:
    A3, B3 = z["A"], z["B"]
with np.load("AB_comp.npz") as zc:
    A4, B4 = zc["A"], zc["B"]

print(np.allclose(A, A2), np.allclose(A, A3), np.allclose(B, B3), np.allclose(A, A4), np.allclose(B, B4))


-rw-rw-r-- 1 uumami uumami 6.9M Oct 22 18:13 AB_comp.npz
-rw-rw-r-- 1 uumami uumami 7.7M Oct 22 18:13 AB.npz
-rw-rw-r-- 1 uumami uumami 3.9M Oct 22 18:13 A.npy
True True True True True


### Qué hace esta celda (inspección de formatos NumPy)

- Genera arreglos `.npy` y archivos `.npz` (con y sin compresión) y muestra sus tamaños.
- Carga de vuelta para verificar que los datos coinciden.
- Demuestra que `.npy`/`.npz` no son legibles por humanos (binario/zip), a diferencia de JSON.



In [42]:
# Inspección de NPY/NPZ: tipo y contenido parcial
!file A.npy AB.npz AB_comp.npz

print("\n--- A.npy hexdump head (64 bytes) ---")
!hexdump -C -n 64 A.npy | head -n 10

# NPZ es un ZIP: listar entradas
print("\n--- AB.npz listado ---")
!unzip -l AB.npz
print("\n--- AB_comp.npz listado ---")
!unzip -l AB_comp.npz


A.npy:       NumPy array, version 1.0, header length 118
AB.npz:      Zip archive data, at least v2.0 to extract, compression method=store
AB_comp.npz: Zip archive data, at least v2.0 to extract, compression method=deflate

--- A.npy hexdump head (64 bytes) ---
00000000  93 4e 55 4d 50 59 01 00  76 00 7b 27 64 65 73 63  |.NUMPY..v.{'desc|
00000010  72 27 3a 20 27 3c 66 34  27 2c 20 27 66 6f 72 74  |r': '<f4', 'fort|
00000020  72 61 6e 5f 6f 72 64 65  72 27 3a 20 46 61 6c 73  |ran_order': Fals|
00000030  65 2c 20 27 73 68 61 70  65 27 3a 20 28 31 30 30  |e, 'shape': (100|
00000040

--- AB.npz listado ---
Archive:  AB.npz
  Length      Date    Time    Name
---------  ---------- -----   ----
  4000128  1980-01-01 00:00   A.npy
  4000128  1980-01-01 00:00   B.npy
---------                     -------
  8000256                     2 files

--- AB_comp.npz listado ---
Archive:  AB_comp.npz
  Length      Date    Time    Name
---------  ---------- -----   ----
  4000128  1980-01-01 00:00   A.n